# 성연령별 Recommender-system  
* 대분류별, 클러스터별

In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## 0. 데이터 로드

In [2]:
cd /

/


In [3]:
cd content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


### 0-1. `qty` and `cnt`

In [25]:
buy_full = pd.read_csv('../data/data.csv', encoding='cp949')

In [28]:
buy_full['date'] = buy_full['date'].apply(lambda x : pd.to_datetime(x, format='%Y%m%d'))
buy_full['year'] = buy_full['date'].apply(lambda x : x.year)
buy_full['month'] = buy_full['date'].apply(lambda x : x.month)
buy_full['day'] = buy_full['date'].apply(lambda x : x.day)

In [29]:
buy_full.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,year,month,day
0,2018-01-01,F,20,식품,가공란,37,0.480964,2018,1,1
1,2018-01-01,F,30,식품,가공란,16,0.480964,2018,1,1
2,2018-01-01,F,40,식품,가공란,9,0.480964,2018,1,1
3,2018-01-01,F,50,식품,가공란,3,0.480964,2018,1,1
4,2018-01-01,M,20,식품,가공란,13,0.480964,2018,1,1


### 0-2. cluster

In [9]:
buy_3_dtw = pd.read_csv('../data/food_tsclustering_norm_3.csv', encoding='CP949')
cluster = buy_3_dtw[['sm_cat', 'cluster']]

In [10]:
cluster.head()

,sm_cat,cluster
0,가공란,2
1,가자미,1
2,갈비/찜/바비큐용 돈육,1
3,갈비용 우육,0
4,갈치,0


### 0-3. 클러스터별 구매이력

In [30]:
data = buy_full.merge(cluster, on='sm_cat')
data.isnull().sum()

date       0
sex        0
age        0
big_cat    0
sm_cat     0
qty        0
cnt        0
year       0
month      0
day        0
cluster    0
dtype: int64

In [31]:
data.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,year,month,day,cluster
0,2018-01-01,F,20,식품,가공란,37,0.480964,2018,1,1,2
1,2018-01-01,F,30,식품,가공란,16,0.480964,2018,1,1,2
2,2018-01-01,F,40,식품,가공란,9,0.480964,2018,1,1,2
3,2018-01-01,F,50,식품,가공란,3,0.480964,2018,1,1,2
4,2018-01-01,M,20,식품,가공란,13,0.480964,2018,1,1,2


In [35]:
data0 = data[data.cluster==0].drop(['big_cat', 'cluster'], axis=1)
data1 = data[data.cluster==1].drop(['big_cat', 'cluster'], axis=1)
data2 = data[data.cluster==2].drop(['big_cat', 'cluster'], axis=1)

### 0-4. 일별 데이터 → 월별 데이터

In [40]:
dat0 = data0.groupby(['year', 'month', 'age', 'sex', 'sm_cat']).sum().reset_index().drop(['day', 'cnt'], axis=1)
dat1 = data1.groupby(['year', 'month', 'age', 'sex', 'sm_cat']).sum().reset_index().drop(['day', 'cnt'], axis=1)
dat2 = data2.groupby(['year', 'month', 'age', 'sex', 'sm_cat']).sum().reset_index().drop(['day', 'cnt'], axis=1)

In [41]:
dat0.head()

,year,month,age,sex,sm_cat,qty
0,2018,1,20,F,갈비용 우육,3
1,2018,1,20,F,갈치,123
2,2018,1,20,F,감/홍시,66
3,2018,1,20,F,감귤/한라봉/오렌지,2162
4,2018,1,20,F,감말랭이,50


## 1. 추천시스템 구현  
> 1) Collaborative Filtering - Item based  
> 2) Matrix Factorization

### 1-1. user_time_index (총 8글자) 
* year : 2018년이면 `01`, 2019년이면 `02`. 
* month : 두글자 숫자 (ex. 1월 `01`)  
* age : `age` 그대로  
* sex : 여자면 `01`, 남자면 `02`

In [51]:
def user_time_id(data):
    id = ''
    # year
    if data['year'] == 2018:
        id += '01'
    else:
        id += '02'
    # month
    id += str(data['month']).zfill(2)
    # age
    id += str(data['age'])
    # sex
    if data['sex'] == 'F':
        id += '01'
    else:
        id += '02'
    return id

In [53]:
dat0['ut_id'] = dat0.apply(user_time_id, axis=1)
dat1['ut_id'] = dat1.apply(user_time_id, axis=1)
dat2['ut_id'] = dat2.apply(user_time_id, axis=1)

In [88]:
item_user0 = pd.pivot_table(dat0, values='qty', index='sm_cat', columns='ut_id').fillna(0)

* 57개의 `sm_cat`  
* 2년치 * 12개월치 * 성별(2categories) * 연령(5categories) = 240개


In [89]:
print(item_user0.shape)
item_user0.head()

(57, 240)


ut_id,01012001,01012002,01013001,01013002,01014001,01014002,01015001,01015002,01016001,01016002,01022001,01022002,01023001,01023002,01024001,01024002,01025001,01025002,01026001,01026002,01032001,01032002,01033001,01033002,01034001,01034002,01035001,01035002,01036001,01036002,01042001,01042002,01043001,01043002,01044001,01044002,01045001,01045002,01046001,01046002,...,02092001,02092002,02093001,02093002,02094001,02094002,02095001,02095002,02096001,02096002,02102001,02102002,02103001,02103002,02104001,02104002,02105001,02105002,02106001,02106002,02112001,02112002,02113001,02113002,02114001,02114002,02115001,02115002,02116001,02116002,02122001,02122002,02123001,02123002,02124001,02124002,02125001,02125002,02126001,02126002
sm_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
갈비용 우육,3.0,1.0,19.0,10.0,26.0,19.0,21.0,16.0,5.0,4.0,2.0,10.0,16.0,24.0,19.0,15.0,16.0,9.0,3.0,1.0,0.0,3.0,6.0,4.0,7.0,18.0,4.0,2.0,2.0,1.0,2.0,2.0,4.0,8.0,2.0,5.0,5.0,2.0,0.0,0.0,...,37.0,44.0,132.0,68.0,55.0,28.0,14.0,12.0,15.0,3.0,53.0,34.0,130.0,100.0,48.0,39.0,7.0,14.0,2.0,3.0,57.0,34.0,113.0,81.0,56.0,39.0,17.0,9.0,7.0,3.0,57.0,25.0,151.0,95.0,76.0,43.0,13.0,12.0,5.0,4.0
갈치,123.0,40.0,930.0,239.0,1073.0,325.0,538.0,255.0,128.0,72.0,139.0,41.0,784.0,259.0,905.0,363.0,502.0,241.0,133.0,102.0,121.0,51.0,646.0,206.0,650.0,265.0,384.0,161.0,107.0,62.0,99.0,47.0,468.0,159.0,577.0,197.0,319.0,133.0,91.0,30.0,...,225.0,62.0,1143.0,385.0,1615.0,516.0,1139.0,358.0,305.0,133.0,185.0,57.0,799.0,273.0,1086.0,406.0,725.0,242.0,206.0,96.0,131.0,34.0,654.0,245.0,700.0,304.0,476.0,182.0,128.0,89.0,177.0,58.0,783.0,232.0,895.0,335.0,536.0,191.0,162.0,88.0
감/홍시,66.0,15.0,267.0,105.0,333.0,176.0,182.0,91.0,52.0,41.0,59.0,11.0,131.0,32.0,139.0,100.0,59.0,32.0,16.0,28.0,52.0,16.0,88.0,36.0,94.0,44.0,45.0,29.0,13.0,15.0,32.0,9.0,63.0,34.0,97.0,29.0,65.0,22.0,15.0,9.0,...,46.0,18.0,95.0,44.0,140.0,74.0,45.0,36.0,11.0,12.0,116.0,23.0,415.0,143.0,818.0,371.0,460.0,209.0,120.0,100.0,111.0,38.0,409.0,150.0,810.0,324.0,626.0,300.0,209.0,140.0,77.0,13.0,206.0,82.0,310.0,128.0,225.0,112.0,67.0,48.0
감귤/한라봉/오렌지,2162.0,776.0,7854.0,2924.0,9480.0,3940.0,4542.0,1958.0,854.0,539.0,1947.0,648.0,7565.0,3025.0,9466.0,4050.0,4569.0,2107.0,872.0,616.0,1441.0,443.0,5889.0,2007.0,8167.0,2872.0,4108.0,1437.0,751.0,380.0,910.0,314.0,3235.0,1252.0,3963.0,1643.0,2056.0,777.0,382.0,227.0,...,1138.0,312.0,5192.0,1779.0,5523.0,2627.0,1885.0,882.0,455.0,290.0,1860.0,539.0,7373.0,2918.0,9229.0,4698.0,3510.0,1767.0,815.0,559.0,3074.0,1046.0,8195.0,3761.0,9508.0,5613.0,3982.0,2255.0,960.0,896.0,2989.0,1089.0,8903.0,3951.0,11107.0,6147.0,5458.0,2801.0,1330.0,981.0
감말랭이,50.0,11.0,217.0,69.0,277.0,108.0,122.0,54.0,24.0,25.0,44.0,6.0,158.0,53.0,212.0,91.0,98.0,50.0,13.0,26.0,18.0,3.0,79.0,21.0,91.0,35.0,44.0,22.0,14.0,11.0,15.0,2.0,49.0,13.0,78.0,24.0,36.0,13.0,13.0,2.0,...,11.0,7.0,63.0,41.0,94.0,43.0,26.0,17.0,5.0,7.0,24.0,6.0,121.0,59.0,145.0,62.0,50.0,37.0,9.0,5.0,32.0,7.0,120.0,50.0,127.0,58.0,44.0,45.0,17.0,17.0,50.0,16.0,209.0,88.0,220.0,95.0,71.0,47.0,25.0,16.0


### 1-2. Collaborative Filtering - Item based   
* `buy_qty` 의 코사인 유사도 계산

In [66]:
from sklearn.metrics.pairwise import cosine_similarity

In [90]:
cosine_similarity0 = cosine_similarity(item_user0)
item0_collabor = pd.DataFrame(data=cosine_similarity0, index=item0.index, columns=item0.index)
print(item0_collabor.shape)
item0_collabor.head()

(57, 57)


sm_cat,갈비용 우육,갈치,감/홍시,감귤/한라봉/오렌지,감말랭이,감자,건대추,건어물 멸치,건어물 황태,게장류,견과류 밤,곡물차,곶감/반건시,과일세트,국내산 돈육,굴 생물,굴비/조기,대게/킹크랩,돼지 곱창,딸기/복분자/블루베리,랍스타,명태/동태,무/배추,미나리,밀크티/티라떼,생선류,소고기 등심/안심,소고기 육회,수산 생물,시금치,양념우육,옥돔,옥수수,유자차,율무차,이온음료,전복 생물,전통주,절임배추/김치속,조개,차 선물세트,참외/메론/수박,코코아/핫초코,탄산음료,토마토,표고버섯,프로폴리스/로얄젤리,한우육,해초류,혼합견과,혼합곡,홍삼 분말/환,홍삼 음료,홍삼/인삼 제품,홍삼액/홍삼정,홍삼절편/홍삼정과,회
sm_cat,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
갈비용 우육,1.000000,0.676245,0.481653,0.673761,0.621847,0.748590,0.625943,0.687050,0.755261,0.844560,0.715639,0.785742,0.390612,0.353629,0.845692,0.523047,0.577241,0.857516,0.873200,0.740177,0.625600,0.834395,0.759693,0.574375,0.778608,0.846131,0.786471,0.900706,0.785660,0.762491,0.904591,0.418313,0.493509,0.680903,0.684826,0.718291,0.606810,0.576696,0.260411,0.782624,0.680893,0.522913,0.797383,0.699948,0.620399,0.735485,0.824517,0.784359,0.578042,0.690067,0.557275,0.643645,0.652503,0.679378,0.708826,0.687688,0.832314
갈치,0.676245,1.000000,0.689495,0.809774,0.820209,0.798330,0.836411,0.958871,0.955268,0.807368,0.888330,0.871660,0.647113,0.610417,0.850242,0.574820,0.908335,0.734856,0.613278,0.783544,0.593919,0.871438,0.818219,0.664898,0.593724,0.905469,0.890649,0.681704,0.928567,0.795127,0.768390,0.732025,0.603700,0.743632,0.802823,0.757648,0.937005,0.614819,0.374262,0.826246,0.736018,0.604896,0.776811,0.720355,0.666659,0.934723,0.895595,0.907945,0.750421,0.869092,0.848440,0.900973,0.895889,0.862789,0.919276,0.895833,0.733697
감/홍시,0.481653,0.689495,1.000000,0.749688,0.719102,0.567363,0.908130,0.685450,0.696853,0.708618,0.814918,0.767029,0.350800,0.234481,0.664841,0.762676,0.550507,0.605862,0.465269,0.557012,0.499719,0.670007,0.697487,0.445398,0.447754,0.684418,0.625945,0.522903,0.667643,0.670651,0.500766,0.371604,0.317792,0.777930,0.803087,0.507531,0.582057,0.406860,0.814948,0.757661,0.443189,0.278606,0.688683,0.517990,0.412829,0.653722,0.714014,0.649161,0.656765,0.545658,0.500776,0.574352,0.573362,0.522002,0.594355,0.590118,0.570077
감귤/한라봉/오렌지,0.673761,0.809774,0.749688,1.000000,0.928414,0.724586,0.838554,0.868902,0.888007,0.826215,0.938072,0.894512,0.657040,0.491064,0.826361,0.821378,0.764423,0.848967,0.613612,0.851636,0.806399,0.885924,0.897145,0.728436,0.583333,0.880118,0.863107,0.675587,0.867615,0.881791,0.683747,0.606612,0.294044,0.884614,0.931977,0.593426,0.724680,0.580529,0.486436,0.922977,0.750871,0.336619,0.888667,0.665543,0.688238,0.845462,0.870125,0.843284,0.887779,0.822407,0.759554,0.802807,0.810647,0.769406,0.834036,0.818407,0.745611
감말랭이,0.621847,0.820209,0.719102,0.928414,1.000000,0.712433,0.784533,0.840882,0.863566,0.748871,0.890170,0.843336,0.663035,0.496310,0.778097,0.807809,0.755490,0.813962,0.564177,0.841867,0.745166,0.831181,0.811601,0.618348,0.561475,0.850286,0.841747,0.638186,0.840132,0.833382,0.648224,0.626389,0.336027,0.878962,0.891241,0.593649,0.737496,0.590651,0.442425,0.840919,0.713431,0.347287,0.862587,0.663846,0.571968,0.830912,0.820324,0.817168,0.943065,0.789058,0.752293,0.793906,0.770833,0.762524,0.815771,0.819119,0.713882


* 아이템 A와 코사인 유사도가 높은 10개의 아이템 도출

In [84]:
def get_item_based_collabor(sm_cat):
    return item0_collabor[sm_cat].sort_values(ascending=False)[1:12]

In [85]:
get_item_based_collabor('갈비용 우육')

sm_cat
양념우육          0.904591
소고기 육회        0.900706
돼지 곱창         0.873200
대게/킹크랩        0.857516
생선류           0.846131
국내산 돈육        0.845692
게장류           0.844560
명태/동태         0.834395
회             0.832314
프로폴리스/로얄젤리    0.824517
코코아/핫초코       0.797383
Name: 갈비용 우육, dtype: float64

### 1-3. Matrix Factorization  
#### 1-3-1. 특정 유저와 유사한 유저가 많이 소비한 상품을 추천

In [92]:
from sklearn.decomposition import TruncatedSVD

In [93]:
user_item0 = pd.pivot_table(dat0, values='qty', index='ut_id', columns='sm_cat').fillna(0)
print(user_item0.shape)
user_item0.head()

(240, 57)


sm_cat,갈비용 우육,갈치,감/홍시,감귤/한라봉/오렌지,감말랭이,감자,건대추,건어물 멸치,건어물 황태,게장류,견과류 밤,곡물차,곶감/반건시,과일세트,국내산 돈육,굴 생물,굴비/조기,대게/킹크랩,돼지 곱창,딸기/복분자/블루베리,랍스타,명태/동태,무/배추,미나리,밀크티/티라떼,생선류,소고기 등심/안심,소고기 육회,수산 생물,시금치,양념우육,옥돔,옥수수,유자차,율무차,이온음료,전복 생물,전통주,절임배추/김치속,조개,차 선물세트,참외/메론/수박,코코아/핫초코,탄산음료,토마토,표고버섯,프로폴리스/로얄젤리,한우육,해초류,혼합견과,혼합곡,홍삼 분말/환,홍삼 음료,홍삼/인삼 제품,홍삼액/홍삼정,홍삼절편/홍삼정과,회
ut_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01012001,3.0,123.0,66.0,2162.0,50.0,1089.0,50.0,78.0,86.0,226.0,254.0,146.0,228.0,48.0,94.0,127.0,110.0,57.0,4590.0,415.0,41.0,18.0,22.0,14.0,1951.0,292.0,100.0,984.0,890.0,51.0,210.0,5.0,52.0,405.0,93.0,373.0,249.0,112.0,17.0,256.0,143.0,48.0,707.0,4558.0,459.0,39.0,613.0,360.0,97.0,348.0,3.0,69.0,21.0,339.0,1330.0,270.0,7374.0
01012002,1.0,40.0,15.0,776.0,11.0,676.0,12.0,33.0,27.0,69.0,75.0,39.0,51.0,7.0,37.0,45.0,26.0,23.0,1850.0,119.0,18.0,12.0,8.0,2.0,616.0,134.0,61.0,671.0,382.0,11.0,68.0,0.0,15.0,233.0,36.0,215.0,63.0,94.0,7.0,107.0,38.0,14.0,264.0,3150.0,126.0,17.0,145.0,267.0,24.0,101.0,1.0,23.0,10.0,147.0,546.0,101.0,3604.0
01013001,19.0,930.0,267.0,7854.0,217.0,1600.0,238.0,589.0,459.0,394.0,1647.0,640.0,925.0,134.0,362.0,587.0,447.0,174.0,4931.0,1756.0,76.0,104.0,134.0,54.0,1716.0,980.0,416.0,1201.0,2798.0,463.0,213.0,28.0,265.0,876.0,282.0,404.0,819.0,167.0,30.0,1191.0,375.0,117.0,1068.0,5280.0,1405.0,275.0,1065.0,1443.0,490.0,938.0,29.0,190.0,64.0,1089.0,4607.0,533.0,11089.0
01013002,10.0,239.0,105.0,2924.0,69.0,637.0,85.0,209.0,165.0,165.0,498.0,212.0,321.0,118.0,146.0,273.0,182.0,109.0,2572.0,537.0,49.0,35.0,42.0,14.0,668.0,565.0,189.0,844.0,1247.0,137.0,156.0,28.0,87.0,377.0,146.0,281.0,276.0,184.0,14.0,518.0,150.0,31.0,475.0,4103.0,477.0,131.0,350.0,639.0,145.0,370.0,8.0,79.0,51.0,509.0,2216.0,274.0,5473.0
01014001,26.0,1073.0,333.0,9480.0,277.0,979.0,219.0,615.0,489.0,477.0,1632.0,488.0,1316.0,226.0,270.0,851.0,652.0,122.0,1509.0,985.0,72.0,120.0,77.0,54.0,625.0,809.0,317.0,948.0,2609.0,352.0,103.0,69.0,265.0,688.0,265.0,369.0,901.0,47.0,95.0,1234.0,263.0,309.0,762.0,2413.0,1261.0,241.0,690.0,1084.0,753.0,708.0,42.0,129.0,58.0,958.0,4554.0,499.0,7090.0


In [132]:
# SVD (특이값 분해)
SVD = TruncatedSVD(n_components=10)
matrix = SVD.fit_transform(user_item0)
matrix.shape

(240, 10)

In [133]:
# 피어슨 상관계수
corr = np.corrcoef(matrix)
corr.shape

(240, 240)

In [134]:
user_list = list(user_item0.index)
corr_user = corr[user_list.index('01012001')] # target user
user_recommend = list(user_item0.index[(corr_user>0.9)])[:50] # target user와 가장 비슷한 최대 50명 user

# 50명 user들이 가장 많이 소비한 item 10개 추천
final_recommend = list(user_item0[user_item0.index.isin(user_recommend)].sum().sort_values(ascending=False)[:10].index) 
final_recommend

['회',
 '돼지 곱창',
 '탄산음료',
 '감귤/한라봉/오렌지',
 '홍삼액/홍삼정',
 '소고기 육회',
 '밀크티/티라떼',
 '수산 생물',
 '감자',
 '토마토']

#### 1-3-2. 특정 상품과 유사한 상품 추천

In [117]:
# SVD (특이값 분해)
SVD = TruncatedSVD(n_components=10)
matrix = SVD.fit_transform(item_user0)
matrix.shape

(57, 10)

In [118]:
# 피어슨 상관계수
corr = np.corrcoef(matrix)
corr.shape

(57, 57)

In [130]:
item_list = list(item_user0.index)
corr_item = corr[item_list.index('갈비용 우육')] # target item
item_recommend = list(item_user0.index[(corr_item>0.9)])[1:51] # target item과 가장 유사한 최대 50개 item
item_recommend

['게장류', '국내산 돈육', '대게/킹크랩', '돼지 곱창', '생선류', '소고기 육회', '양념우육', '프로폴리스/로얄젤리']